In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import nept

from loading_data import get_data

In [ ]:
import math

In [ ]:
sig = nept.get_edges(position, np.median(np.diff(position.time)), lastbin=True)
frame_len = 0.02
frame_step = 0.005

slen = len(sig)
if slen <= frame_len:
    numframes = 1
else:
    numframes = 1 + int(math.ceil((1.0 * slen - frame_len) / frame_step))

padlen = int((numframes - 1) * frame_step + frame_len)
padsignal = np.concatenate((sig,  np.zeros(padlen - slen)))

indices = (np.tile(np.arange(frame_len), (numframes, 1))
           + np.tile(np.arange(0, numframes * frame_step, frame_step),
                     (frame_len, 1)).T.astype(dtype=np.int32))
frames = padsignal[indices]
win = np.tile(winfunc(frame_len), (numframes, 1))

yy = frames * win
print(yy)

In [ ]:
binsize = 0.02
stepsize = 0.005

n_counts = binsize / stepsize

newarray = np.array([1])
this_position = nept.Position([np.insert(position.x[0:-len(newarray)], 0, np.array(np.zeros(len(newarray))))],
                               np.insert(position.time[0:-len(newarray)], 0, np.array(np.min(position.time)-newarray)))
edges = nept.get_edges(this_position, stepsize, lastbin=True)

counts = np.zeros((len(spikes), len(edges)-1))
for idx, spiketrain in enumerate(spikes):
    counts[idx] = np.histogram(spiketrain.time, bins=edges)[0]

In [ ]:
position.time

In [ ]:
spikes = [nept.SpikeTrain([0.8, 1.1, 1.2, 1.2, 2.1, 3.1])]
position = nept.Position(np.array([1., 6.]), np.array([0., 4.]))

binsize = 0.02
stepsize = 0.005

def test_binning(position, spikes, binsize=0.025, stepsize=0.0125):
    edges = nept.get_edges(position, stepsize, lastbin=False)
    final_counts = np.zeros((len(spikes), len(edges)-1))

    count = 0
    for num in range(int(binsize / stepsize)):
        count += 1
        these_edges = np.arange(edges[num], edges[-1]+(stepsize*count), stepsize)

        counts = np.zeros((len(spikes), len(these_edges)-1))
        for idx, spiketrain in enumerate(spikes):
            counts[idx] = np.histogram(spiketrain.time, bins=these_edges)[0]
        final_counts += counts
        
    return final_counts

In [ ]:
final_counts

In [ ]:
print(len(counts[0]))

In [ ]:
print(counts)

In [ ]:
int(math.ceil((1.0 * slen - frame_len))

In [ ]:
def framesig(sig, frame_len, frame_step, winfunc=lambda x: np.ones(x)):
    """Frame a signal into overlapping frames.
    Parameters
    ----------
    sig : array_like
        The audio signal to frame.
    frame_len : int
        Length of each frame, in samples.
    frame_step : int
        Number of samples after the start of the previous frame
        that the next frame should begin.
    winfunc : function, optional
        The analysis window to apply to each frame. Default: No window
    Returns
    -------
    Numpy array of frames with shape (NUMFRAMES, frame_len).
    """

    slen = len(sig)
    frame_len = int(round(frame_len))
    frame_step = int(round(frame_step))
    if slen <= frame_len:
        numframes = 1
    else:
        numframes = 1 + int(math.ceil((1.0 * slen - frame_len) / frame_step))

    padlen = int((numframes - 1) * frame_step + frame_len)
    padsignal = np.concatenate((sig,  np.zeros(padlen - slen)))

    indices = (np.tile(np.arange(frame_len), (numframes, 1))
               + np.tile(np.arange(0, numframes * frame_step, frame_step),
                         (frame_len, 1)).T.astype(dtype=np.int32))
    frames = padsignal[indices]
    win = np.tile(winfunc(frame_len), (numframes, 1))
    return frames * win

In [ ]:
import info.r066d3 as info

In [ ]:
home = os.path.expanduser("~")
emi_shortcut = os.path.join(home, "code", "emi_shortcut")
pickle_filepath = os.path.join(emi_shortcut, "cache", "pickled")
# pickle_filepath = 'E:/code/emi_shortcut/cache/pickled'
output_filepath = os.path.join(emi_shortcut, "plots")

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = nept.get_xyedges(position)

In [ ]:
type(position) == nept.objects.Position

# Saving many tuning curves on the same plot

In [ ]:
import matplotlib.gridspec as gridspec

rows = 8
cols = 14
fig = plt.figure(figsize=(10, 10))

grids = gridspec.GridSpec(cols, rows)
grids.update(wspace=0.02, hspace=0.05)

xx, yy = np.meshgrid(xedges, yedges)
for i in range(rows*cols):
    ax = plt.subplot(grids[i])
    pp = ax.pcolormesh(xx, yy, tuning_curves[i], cmap='pink_r')
    ax.axis('off')
    
plt.show()

# savepath = 'E:/code/emi_shortcut/plots/tuning/' + info.session_id + 'many_tuning-curves.png'
# plt.savefig(savepath, dpi=300, bbox_inches='tight')

# Testing Neuralynx events

In [ ]:
dataloc = 'C:/Users/EmilyWork/Desktop/20170104'
session = '2017-01-05_test1'
event_filename = 'Events.nev'
# event_labels = dict(on='on',
#                     off='off')
event_labels = dict(on='TTL Output on AcqSystem1_0 board 0 port 3 value (0x0002).',
                    off='TTL Output on AcqSystem1_0 board 0 port 3 value (0x0000).')

In [ ]:
filepath = os.path.join(dataloc, session, event_filename)
events = nept.load_events(filepath, event_labels)

In [ ]:
len(events['off']), len(events['on'])